In [1]:
!pip install transformers[torch]
!pip install datasets
!pip install sentence_transformers
!pip install numba

import pandas as pd
import numpy as np
from numba import cuda
from transformers import DataCollatorForSeq2Seq
from transformers import AutoModelForSequenceClassification, AutoTokenizer
from datasets import Dataset
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from tqdm import tqdm
import torch


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.4/297.4 kB 6.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [2]:
from google.colab import drive
drive.mount('/content/drive')

df1=pd.read_csv(r'/content/drive/MyDrive/preprocessedexp_dataset.csv')
df1.head()
df1.drop(columns=['File Name','Transcript','tokenized_Transcript'],inplace=True)

df1.head()


Mounted at /content/drive


,Unnamed: 0,Label,clean_text
0,0,truthful,recently visit paris moment feel like scene ro...
1,1,truthful,step fruit u s o time accelerate travel san fr...
2,2,truthful,trip u s incredible landing new york city feel...
3,3,truthful,visit usa time like enter new world mention me...
4,4,truthful,step plane usa time fill anticipation visit wa...


In [3]:
import spacy
from collections import Counter
!python -m spacy download en_core_web_lg
# Load Spacy's English language model
nlp = spacy.load("en_core_web_lg")


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.7/587.7 MB 2.2 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [4]:
# Define a function to tokenize text
def tokenize(txt):
    # Remove punctuation and double space, and convert to lowercase:
    tokens = [token.text.lower() for token in txt if not (token.is_punct or token.is_space)]
    # Join tokens back into a string
    tokens = " ".join(tokens)
    return tokens

# Apply the preprocessing pipeline using nlp.pipe
tokenized_txt = []

for txt in nlp.pipe(df1['clean_text']):
    tokenized_txt.append(tokenize(txt))



In [5]:

# Add the preprocessed text as a new column in the dataframe
df1['tokenized_Transcript'] = tokenized_txt
df1.head()
df1.drop(columns=['clean_text','Unnamed: 0'],inplace=True)
df1.head()
df1.isna().sum()
df1['Label'] = df1['Label'].map({'truthful': 1, 'decpetive': 0})
df1['Label'].fillna(0, inplace=True)
df1['Label'] = df1['Label'].astype(int)
from transformers import T5Tokenizer, T5ForConditionalGeneration
# Preprocess function to tokenize input text and format labels
def preprocess_function(examples):
    inputs = examples["tokenized_Transcript"]
    targets = examples["Label"]

    # Convert labels to lists
    targets = [[label] for label in targets]

    # Ensure that inputs are lists of strings
    if isinstance(inputs, str):
        inputs = [inputs]

    # Tokenize inputs
    inputs_tokenized = tokenizer(inputs, padding=True, truncation=True)['input_ids']

    return {'input_ids': inputs_tokenized, 'labels': targets}




In [6]:
def create_dataset(df1, seed=42, cv=10):
    train = {f'split_{i+1}': [] for i in range(cv)}
    test = {f'split_{i+1}': [] for i in range(cv)}

    np.random.seed(seed)

    for k in train.keys():
        train[k] = df1.sample(frac=0.8, random_state=seed)
        test[k] = df1.drop(train[k].index)

    return train, test


In [7]:
seed = 42
numcv = 10
num_epochs = 3
model_size = 'large'
!pip install accelerate
import accelerate
from sklearn.metrics import accuracy_score
train, test = create_dataset(df1, seed=seed, cv=numcv)
results = {}
collect_result = {}
ground_truth = {}


In [8]:
# Define the device
device = "cuda" if torch.cuda.is_available() else "cpu"
for sp in tqdm(train.keys()):
    data_train = Dataset.from_pandas(train[sp])
    data_test = Dataset.from_pandas(test[sp])
    tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-large")
    model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", device_map="auto")
    # Tokenize and preprocess data
    data_train = data_train.map(preprocess_function, batched=True)
    data_test = data_test.map(preprocess_function, batched=True)

    training_args = Seq2SeqTrainingArguments(
        output_dir="./results",  # The output directory
        logging_dir="./logs",
        evaluation_strategy="epoch",  # Evaluation is done at the end of each epoch
        learning_rate=5e-4,
        per_device_train_batch_size=2,
        weight_decay=0.01,
        save_total_limit=3,
        num_train_epochs=num_epochs,
        predict_with_generate=True,
        fp16=False,
        push_to_hub=False,
    )
    # Define trainer
    trainer = Seq2SeqTrainer(
        model=model,
        args=training_args,
        train_dataset=data_train,
        eval_dataset=data_test,
        tokenizer=tokenizer,
    )
        # Train the model
    trainer.train()
    # Get predictions
    predictions = trainer.predict(data_test)
    pred_labels = np.argmax(predictions.predictions, axis=1)

    # Store predictions in the original DataFrame
    df1.loc[test[sp].index, 'Prediction'] = pred_labels
    df1['Prediction'].fillna(0, inplace=True)

    # Clean up
    del model
    torch.cuda.empty_cache()





  0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

...

Map:   0%|          | 0/205 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,No log,0.641431
2,2.433300,1.423164
3,1.124000,0.653452


Checkpoint destination directory ./results/checkpoint-500 already exists and is non-empty. Saving will proceed but saved results may be invalid.
Checkpoint destination directory ./results/checkpoint-1000 already exists and is non-empty. Saving will proceed but saved results may be invalid.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


100%|██████████| 10/10 [2:07:21<00:00, 764.14s/it]


In [10]:
# Display final DataFrame with predictions
print("Final DataFrame with Predictions:")
print(df1)

# Assuming ground_truth is defined outside the loop
# Calculate accuracy
ground_truth = df1.loc[test[sp].index, 'Label']
test_accuracy = accuracy_score(ground_truth, df1.loc[test[sp].index, 'Prediction'])
print("Test Accuracy:",test_accuracy)

Final DataFrame with Predictions:
      Label                               tokenized_Transcript  Prediction
0         1  recently visit paris moment feel like scene ro...         0.0
1         1  step fruit u s o time accelerate travel san fr...         0.0
2         1  trip u s incredible landing new york city feel...         0.0
3         1  visit usa time like enter new world mention me...         0.0
4         1  step plane usa time fill anticipation visit wa...         0.0
...     ...                                                ...         ...
1021      0  travel experience visit dark hedge northern ir...         0.0
1022      0  travel experience visit muchu picchu peru actu...         0.0
1023      0  travel experience visit hallstatt austria actu...         0.0
1024      0  trip visit jiuzhaigou valley china actually na...         0.0
1025      0  trip visit bagan myanmar actually old city bud...         0.0

[1026 rows x 3 columns]
Test Accuracy: 0.6926829268292682


In [12]:
"""# Display final DataFrame with predictions
print("Final DataFrame with Predictions:")
print(df1)

# Assuming ground_truth is defined outside the loop
# Calculate accuracy
ground_truth = df1.loc[test[sp].index, 'Label']
test_accuracy = accuracy_score(ground_truth, df1.loc[test[sp].index, 'Prediction'])
print("Test Accuracy:", test_accuracy)"""

'# Display final DataFrame with predictions\nprint("Final DataFrame with Predictions:")\nprint(df1)\n\n# Assuming ground_truth is defined outside the loop\n# Calculate accuracy\nground_truth = df1.loc[test[sp].index, \'Label\']\ntest_accuracy = accuracy_score(ground_truth, df1.loc[test[sp].index, \'Prediction\'])\nprint("Test Accuracy:",\xa0test_accuracy)'